### Prepare the data

In [2]:
from tensorflow.keras.preprocessing import text_dataset_from_dictionary

In [ ]:
# Assumes you're in the root level of the dataset directory.
# If you aren't, you'll need to change the relative paths here.
train_data = text_dataset_from_directory("./train")
test_data = text_dataset_from_directory("./test")

In [ ]:
from tensorflow.strings import regex_replace

def prepareData(dir):
    data = text_dataset_from_directory(dir)
    return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )

train_data = prepareData('./train')
test_data = prepareData('./test')

In [ ]:
for text_batch, label_batch in train_data.take(1):
    print(text_batch.numpy()[0])
    print(label_batch.numpy()[0]) # 0 = negative, 1 = positive

### Build the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input

model = Sequential()
model.add(Input(shape = (1,), dtype = 'string'))

#### 3.1 Text Vectorization
Our first layer will be a TextVectorization layer, which will process the input string and turn it into a sequence of integers, each one representing a token.

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  # Max vocab size. Any words outside of the max_tokens most common ones
  # will be treated the same way: as "out of vocabulary" (OOV) tokens.
  max_tokens=max_tokens,
  # Output integer indices, one per string token
  output_mode="int",
  # Always pad or truncate to exactly this many tokens
  output_sequence_length=max_len,
)

In [ ]:
# Call adapt(), which fits the TextVectorization layer to our text dataset.
# This is when the max_tokens most common words (i.e. the vocabulary) are selected.
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

In [ ]:
model.add(vectorize_layer)

#### 3.2 Embedding
Our next layer will be an Embedding layer, which will turn the integers produced by the previous layer into fixed-length vectors.

In [ ]:
from tensorflow.keras.layers import Embedding 

In [ ]:
# Note that we're using max_tokens + 1 here, since there's an
# out-of-vocabulary (OOV) token that gets added to the vocab.
model.add(Embedding(max_tokens + 1, 128))

#### 3.3 The Recurrent Layer
Finally, we’re ready for the recurrent layer that makes our network a RNN! We’ll use a Long Short-Term Memory (LSTM) layer, which is a popular choice for this kind of problem. It’s very simple to implement:

In [ ]:
from tensorflow.keras.layers import LSTM

In [ ]:
model.add(LSTM(64))

To finish off our network, we’ll add a standard fully-connected (Dense) layer and an output layer with sigmoid activation:

In [ ]:
from tensorflow.keras.layers import Dense

In [ ]:
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

### 4. Compiling the Model
Before we can begin training, we need to configure the training process. We decide a few key factors during the compilation step, including:

* The optimizer. We’ll stick with a pretty good default: the Adam gradient-based optimizer. Keras has many other optimizers you can look into as well.
* The loss function. Since we only have 2 output classes (positive and negative), we’ll use the Binary Cross-Entropy loss. See all Keras losses.
* A list of metrics. Since this is a classification problem, we’ll just have Keras report on the accuracy metric.

In [ ]:
model.compile( optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

### 5. Training the Model

In [ ]:
model.fit(train_data, epoches = 10)

### 6. Using the Model

In [ ]:
model.save_weights('lstm.h5')

In [ ]:
model = Sequential()
model.load_weights('lstm.h5')


In [ ]:
print(model.predit(['i love it! highly recommend it to anyone and everyone']))

### 7. Extensions
* Network Depth: we can add one more LSTM layer but need to make sure that we output the full sequence instead of only the last output - model.add(LSTM(64, return_sequences=True))
* Dropout - like below
* Text Vectorization Parameters - The max_tokens and max_len parameters used in our TextVectorization layer are natural candidates for tinkering
* Pre-processing on the text e.g. remove unnecessary tokens

In [ ]:
# Examples of common ways to use dropout below. These
# parameters are not necessarily the most optimal.
model.add(LSTM(64, dropout=0.25, recurrent_dropout=0.25))

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))